In [1]:
import itertools
import pickle as pickle
import plotly.graph_objects as go
from scipy.ndimage.filters import gaussian_filter1d
from torch_geometric.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


from analysis.graphanalysis import *

# Loading the data

In [2]:
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='r', folder='data/processed/')

  3%|███▍                                                                                                              | 17/557 [00:00<00:03, 163.62it/s]

Reading files with generic numbers on receptors.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [00:03<00:00, 156.99it/s]


# Create graph analysis class

In [3]:
gp = GraphProcessor(d=[], p=p)

Reading data from data/couplings/families_coupling.xls!


C:\Users\Sideadmin\.conda\envs\mt\lib\site-packages\pandas\core\generic.py:4152: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Initialized Affinity Processor!
Please set a group --------------  ['GPCRdb', 'Inoue', 'Bouvier'].
please set label type -----------  ['Guide to Pharmacology', 'Log(Emax/EC50)', 'pEC50', 'Emax'].

Selected label type 'Log(Emax/EC50)'.


Selected data of group 'GPCRdb'.



In [4]:
gp.set_atom_list()
gp.apply_atom_list_filter()

In [5]:
gp.simplify()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 368/368 [00:02<00:00, 138.03it/s]


# Create training settings

In [127]:
hyper_params = {
    'learning_rate': [0.00001],
    'cons_r_res':[
        ['3.53', '7.55'],
        ['3.53', '6.37'],
        ['6.37', '7.55'],
        ['2.55', '3.53', '7.53']
        
    ],
    'radius': [10, 13, 16],
    'max_edge_dist': [7, 9],
    'batch_size': [2, 8]
}

In [128]:
keys, values = zip(*hyper_params.items())
keys = tuple(['index']) + keys
configs = [dict(zip(keys, [idx, *v])) for idx, v in enumerate(itertools.product(*values))]

In [129]:
len(configs)

48

# Run Analysis

In [138]:
def run_training(model, n_epochs, lr, train_loader, validation_loader, patience = 50):
    n_outputs=4
    h5n = H5Net(n_outputs=n_outputs, aggr='add')
    h5n.to('cuda')
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(h5n.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=patience, 
                                                           threshold=0.0001, threshold_mode='rel', 
                                                           cooldown=0, min_lr=0, eps=1e-8, verbose=True)

    print("Initialized model!")
    n_epochs = n_epochs

    t_e_losses = []
    t_mse_losses = []
    v_e_losses = []
    v_mse_losses = []

    best_validation = 12
    best_training = 12
    
    for e in range(n_epochs):
        # TRAINING
        training_loss = 0
        t_bar = tqdm(enumerate(train_loader), desc='TRAINING')
        for b,  batch in t_bar:
            batch.z = batch.x
            batch.to('cuda')
            pred = h5n(batch)
            target = batch.y / 12  # very crude normalization! (should substract upper & lower limit etc)
            target = target.reshape(-1, n_outputs)
            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()
            training_loss = (training_loss * b + loss) / (b + 1)
            if b % 5 == 0:
                t_bar.set_description(model + ' | E: {} | TRAINING: MSE = '.format(e)
                                      +str(round(float(training_loss), 6)))
        
        t_mse_loss = training_loss.cpu().detach().numpy()
        t_e_loss = round(torch.sqrt(training_loss).cpu().detach().numpy() * 12, 2)
        t_e_losses.append(t_e_loss) # take the square root to get the mean error, then *12
        t_mse_losses.append(t_mse_loss)
        
        if t_e_loss < best_training:
            best_training = t_e_loss            
        
        # VALIDATION
        with torch.no_grad():
            validation_loss = 0
            t_bar = tqdm(enumerate(validation_loader), desc='VALIDATION')
            for b,  batch in t_bar:
                batch.z = batch.x
                batch.to('cuda')
                pred = h5n(batch)
                target = batch.y / 12
                target = target.reshape(-1, n_outputs)
                loss = criterion(pred, target)
                validation_loss = (validation_loss * b + loss) / (b + 1)
                if b % 5 == 0:
                    t_bar.set_description(model + ' | E: {} | VALIDATION: MSE = '.format(e)
                                          +str(round(float(validation_loss), 6)))
        scheduler.step(validation_loss)
        v_mse_loss = validation_loss.cpu().detach().numpy()
        v_e_loss = round(torch.sqrt(validation_loss).cpu().detach().numpy() * 12, 2)
        v_e_losses.append(v_e_loss)
        v_mse_losses.append(v_mse_loss)
        
        if v_e_loss < best_validation:
            print("New best validation perfomance: MSE={} | mean epoch error={}".format(v_mse_loss, v_e_loss))
            best_validation = v_e_loss
            ckpt = h5n.state_dict()
            last_update = e
        
        if e - (4 * patience) > last_update:
            print("STOP (no further improvement recorded after {} epochs)".format(4 * patience))
            break       
            
    return ckpt, t_e_losses, t_mse_losses, best_training, v_e_losses, v_mse_losses, best_validation

In [139]:
from torch_geometric.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


def run(model, gp, config, performance, n_epochs=250, save=True):
    gp.create_graph(filter_by_chain=True,
                gpcr=True,
                gprotein=False,
                auxilary=False,
                node_criteria='Interaction Site', 
                edge_criteria='radius',
                h5start=13,
                cons_r_res=config['cons_r_res'], 
                radius=config['radius'],
                max_edge_dist=config['max_edge_dist'])
    print("Finished creating Graphs!")
    validation_split = .2
    shuffle_dataset = True

    # Creating data indices for training and validation splits
    dataset_size = len(gp)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    
    print("Initializing DataLoader ==> Batchsize: {}".format(config['batch_size']))
    train_loader = DataLoader(gp, batch_size=int(config['batch_size']), sampler=train_sampler)
    validation_loader = DataLoader(gp, batch_size=int(config['batch_size']), sampler=valid_sampler)
    
    print("Starting Training ({} Epochs)!".format(n_epochs))
    ckpt, t_e_losses, t_mse_losses, best_training, v_e_losses, v_mse_losses, best_validation = run_training(
        model=model,
        n_epochs=n_epochs, 
        lr = config['learning_rate'],
        train_loader=train_loader, 
        validation_loader=validation_loader)
    performance[config['index']] = {
        'best_training': best_training,
        'training_epoch_losses': t_e_losses,
        'training_mse_losses': t_mse_losses,
        'best_validation': best_validation,
        'validation_epoch_losses': v_e_losses,
        'validation_mse_losses': v_mse_losses
    }
    if save:
        picklefile = open(str('models/performances'+model+'.pkl'),'wb')
        pickle.dump(performance, picklefile)
    return performance, ckpt

In [140]:
performance = {}

for c, config in enumerate(configs):
    if c in top5:
        values = list(config.values())
        model = 'model_' + "_".join([str(x) if not isinstance(x, list) else "_".\
                                     join([str(round(float(y), 2)).replace(".", "") for y in x]) for x in values])
        np.random.seed(seed=c)
        torch.manual_seed(c)
        print("\n\n\nRUNNING NEW CONFIGURATION:\n",config)
        performance, ckpt = run(model, gp, config, performance, n_epochs=1000)

  0%|▎                                                                                                                   | 1/368 [00:00<00:40,  9.09it/s]




RUNNING NEW CONFIGURATION:
 {'index': 0, 'learning_rate': 1e-05, 'cons_r_res': ['3.53', '7.55'], 'radius': 10, 'max_edge_dist': 7, 'batch_size': 2}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 368/368 [00:15<00:00, 24.28it/s]
model_0_1e-05_353_755_10_7_2 | E: 0 | TRAINING: MSE = 0.129826: : 11it [00:00, 107.82it/s]

Finished creating Graphs!
Initializing DataLoader ==> Batchsize: 2
Starting Training (1000 Epochs)!
Initialized model!


model_0_1e-05_353_755_10_7_2 | E: 0 | TRAINING: MSE = 0.122669: : 80it [00:00, 109.87it/s]
model_0_1e-05_353_755_10_7_2 | E: 0 | VALIDATION: MSE = 0.10798: : 20it [00:00, 312.43it/s]
model_0_1e-05_353_755_10_7_2 | E: 1 | TRAINING: MSE = 0.108878: : 11it [00:00, 108.89it/s]

...

model_28_1e-05_637_755_13_7_2 | E: 443 | TRAINING: MSE = 0.026173: : 87it [00:00, 107.65it/s]
model_28_1e-05_637_755_13_7_2 | E: 443 | VALIDATION: MSE = 0.051816: : 22it [00:00, 313.58it/s]
model_28_1e-05_637_755_13_7_2 | E: 444 | TRAINING: MSE = 0.026194: : 87it [00:00, 103.77it/s]
model_28_1e-05_637_755_13_7_2 | E: 444 | VALIDATION: MSE = 0.056198: : 22it [00:00, 323.22it/s]
model_28_1e-05_637_755_13_7_2 | E: 445 | TRAINING: MSE = 0.026641: : 87it [00:00, 105.58it/s]
model_28_1e-05_637_755_13_7_2 | E: 445 | VALIDATION: MSE = 0.057465: : 22it [00:00, 305.49it/s]
model_28_1e-05_637_755_13_7_2 | E: 446 | TRAINING: MSE = 0.026351: : 87it [00:00, 106.36it/s]
model_28_1e-05_637_755_13_7_2 | E: 446 | VALIDATION: MSE = 0.056121: : 22it [00:00, 328.53it/s]
model_28_1e-05_637_755_13_7_2 | E: 447 | TRAINING: MSE = 0.026032: : 87it [00:00, 108.48it/s]
model_28_1e-05_637_755_13_7_2 | E: 447 | VALIDATION: MSE = 0.056579: : 22it [00:00, 318.77it/s]
model_28_1e-05_637_755_13_7_2 | E: 448 | TRAINING:

Epoch   451: reducing learning rate of group 0 to 8.0000e-08.


model_28_1e-05_637_755_13_7_2 | E: 451 | TRAINING: MSE = 0.027719: : 87it [00:00, 106.33it/s]
model_28_1e-05_637_755_13_7_2 | E: 451 | VALIDATION: MSE = 0.054587: : 22it [00:00, 318.77it/s]
model_28_1e-05_637_755_13_7_2 | E: 452 | TRAINING: MSE = 0.026387: : 87it [00:00, 107.25it/s]
model_28_1e-05_637_755_13_7_2 | E: 452 | VALIDATION: MSE = 0.056245: : 22it [00:00, 318.77it/s]
model_28_1e-05_637_755_13_7_2 | E: 453 | TRAINING: MSE = 0.026534: : 87it [00:00, 108.18it/s]
model_28_1e-05_637_755_13_7_2 | E: 453 | VALIDATION: MSE = 0.054601: : 22it [00:00, 305.48it/s]
model_28_1e-05_637_755_13_7_2 | E: 454 | TRAINING: MSE = 0.025791: : 87it [00:00, 107.65it/s]
model_28_1e-05_637_755_13_7_2 | E: 454 | VALIDATION: MSE = 0.057306: : 22it [00:00, 318.77it/s]
model_28_1e-05_637_755_13_7_2 | E: 455 | TRAINING: MSE = 0.026422: : 87it [00:00, 105.05it/s]
model_28_1e-05_637_755_13_7_2 | E: 455 | VALIDATION: MSE = 0.05776: : 22it [00:00, 314.22it/s]
model_28_1e-05_637_755_13_7_2 | E: 456 | TRAINING: 

model_28_1e-05_637_755_13_7_2 | E: 494 | TRAINING: MSE = 0.025744: : 87it [00:00, 104.31it/s]
model_28_1e-05_637_755_13_7_2 | E: 494 | VALIDATION: MSE = 0.064118: : 22it [00:00, 244.39it/s]
model_28_1e-05_637_755_13_7_2 | E: 495 | TRAINING: MSE = 0.026853: : 87it [00:00, 92.69it/s]
model_28_1e-05_637_755_13_7_2 | E: 495 | VALIDATION: MSE = 0.053378: : 22it [00:00, 309.79it/s]
model_28_1e-05_637_755_13_7_2 | E: 496 | TRAINING: MSE = 0.025144: : 87it [00:00, 108.05it/s]
model_28_1e-05_637_755_13_7_2 | E: 496 | VALIDATION: MSE = 0.056731: : 22it [00:00, 328.29it/s]
model_28_1e-05_637_755_13_7_2 | E: 497 | TRAINING: MSE = 0.026665: : 87it [00:00, 114.15it/s]
model_28_1e-05_637_755_13_7_2 | E: 497 | VALIDATION: MSE = 0.058303: : 22it [00:00, 349.13it/s]
model_28_1e-05_637_755_13_7_2 | E: 498 | TRAINING: MSE = 0.026698: : 87it [00:00, 110.38it/s]
model_28_1e-05_637_755_13_7_2 | E: 498 | VALIDATION: MSE = 0.069154: : 22it [00:00, 323.46it/s]

STOP (no further improvement recorded after 200 epochs)


In [141]:
picklefile = open(str('models/performances.pkl'),'wb')

In [142]:
pickle.dump(performance, picklefile)

In [176]:
with open("models/performances.pkl", "rb") as input_file:
    data = pickle.load(input_file)

# Visualization

In [187]:

from plotly.subplots import make_subplots

def loss_plot(performance, config, save=False):
    values = list(config.values())
    model = 'model_' + "_".join([str(x) if not isinstance(x, list) else "_".\
                                 join([str(round(float(y), 2)).replace(".", "") for y in x]) for x in values])
    n_epochs = len(performance['validation_epoch_losses'])
    def make_plot(fig, a, b, col=0, loss='MSE'):
        # Create traces
        fig.add_trace(go.Scatter(
            y=a,
            mode='lines',
            name='Validation loss'))

        fig.add_trace(go.Scatter(
            y=b,
            mode='lines',
            name='Training loss'))

        y_title = loss + ' [log(Emax/EC50)]'
        fig.update_layout(
            title=model,
            xaxis_title='Epoch',
            yaxis_title=y_title
            )
        
        vysmoothed = gaussian_filter1d(a, sigma=2)
        fig.add_trace(go.Scatter(y=vysmoothed))
        
        tysmoothed = gaussian_filter1d(b, sigma=2)
        fig.add_trace(go.Scatter(y=tysmoothed))
        return fig
    fig = go.Figure()
    fig = make_plot(fig, performance['validation_epoch_losses'], performance['training_epoch_losses'], loss='Mean Absolute Error (MAE) \n')
    fig.show()
    if save:
        fig.write_image("plots/" + model + "mae_.png")
    del(fig)
    fig = go.Figure()
    fig = make_plot(fig, performance['validation_mse_losses'], performance['training_mse_losses'], loss='Mean Squared Error (MSE)')
    fig.show()
    if save:
        fig.write_image("plots/" + model + "_mse.png")

In [188]:
for i in list(data.keys()):
    loss_plot(data[i], config=configs[i], save=True)

In [175]:
def get_best_config(performance):
    best_vals = []
    idxs = []
    for i, p in enumerate(performance.values()):
        best_vals.append(round(float(p['best_validation'].cpu().numpy()), 3))
        idxs.append(i)
    return [(y, x) for (x, y) in list(sorted(zip(best_vals, idxs)))]

In [63]:
top5 = list(dict(get_best_config(performance)[:5]).keys())

In [65]:
top5

[26, 28, 0, 12, 2]

In [66]:
[configs[x] for x in top5]

[{'index': 26,
  'learning_rate': 1e-05,
  'cons_r_res': ['6.37', '7.55'],
  'radius': 10,
  'max_edge_dist': 9,
  'batch_size': 2},
 {'index': 28,
  'learning_rate': 1e-05,
  'cons_r_res': ['6.37', '7.55'],
  'radius': 13,
  'max_edge_dist': 7,
  'batch_size': 2},
 {'index': 0,
  'learning_rate': 1e-05,
  'cons_r_res': ['3.53', '7.55'],
  'radius': 10,
  'max_edge_dist': 7,
  'batch_size': 2},
 {'index': 12,
  'learning_rate': 1e-05,
  'cons_r_res': ['3.53', '6.37'],
  'radius': 10,
  'max_edge_dist': 7,
  'batch_size': 2},
 {'index': 2,
  'learning_rate': 1e-05,
  'cons_r_res': ['3.53', '7.55'],
  'radius': 10,
  'max_edge_dist': 9,
  'batch_size': 2}]